In [1]:
import pandas as pd


In [2]:
ratings_small = pd.read_csv(fr'ratings_small.csv')
yt_df = pd.read_csv(fr'ml-youtube.csv')


In [3]:
ratings_small

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [4]:
unique_movie_ids_from_ratings_small = ratings_small['movieId'].unique().tolist()
len(unique_movie_ids_from_ratings_small)

9066

In [5]:
yt_df

,youtubeId,movieId,title
0,K26_sDKnvMU,1,Toy Story (1995)
1,3LPANjHlPxo,2,Jumanji (1995)
2,rEnOoWs3FuA,3,Grumpier Old Men (1995)
3,j9xml1CxgXI,4,Waiting to Exhale (1995)
4,ltwvKLnj1B4,5,Father of the Bride Part II (1995)
...,...,...,...
25618,-oB6DN5dYWo,131252,Forklift Driver Klaus: The First Day on the Jo...
25619,DK7KQ-gEdl4,131256,"Feuer, Eis & Dosenbier (2002)"
25620,v29P-wchMZQ,131258,The Pirates (2014)
25621,dAz-nZ65jYU,131260,Rentun Ruusu (2001)


In [6]:
yt_df_filtered = yt_df[yt_df['movieId'].isin(list(unique_movie_ids_from_ratings_small))]
yt_df_filtered

,youtubeId,movieId,title
0,K26_sDKnvMU,1,Toy Story (1995)
1,3LPANjHlPxo,2,Jumanji (1995)
2,rEnOoWs3FuA,3,Grumpier Old Men (1995)
3,j9xml1CxgXI,4,Waiting to Exhale (1995)
4,ltwvKLnj1B4,5,Father of the Bride Part II (1995)
...,...,...,...
25535,B6uQ5MMlx3g,130960,The Conrad Boys (2006)
25538,qCD9BG7ioOc,130970,George Carlin: Life Is Worth Losing (2005)
25541,zElDJ1f3Jk0,130980,The Day That Lasted 21 Years (2012)
25548,lEqrpuU9fYI,131013,Get Hard (2015)


In [7]:
yt_df_filtered.reset_index(drop=True, inplace=True)
yt_df_filtered

,youtubeId,movieId,title
0,K26_sDKnvMU,1,Toy Story (1995)
1,3LPANjHlPxo,2,Jumanji (1995)
2,rEnOoWs3FuA,3,Grumpier Old Men (1995)
3,j9xml1CxgXI,4,Waiting to Exhale (1995)
4,ltwvKLnj1B4,5,Father of the Bride Part II (1995)
...,...,...,...
8687,B6uQ5MMlx3g,130960,The Conrad Boys (2006)
8688,qCD9BG7ioOc,130970,George Carlin: Life Is Worth Losing (2005)
8689,zElDJ1f3Jk0,130980,The Day That Lasted 21 Years (2012)
8690,lEqrpuU9fYI,131013,Get Hard (2015)


In [8]:
yt_df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8692 entries, 0 to 8691
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   youtubeId  8692 non-null   object
 1   movieId    8692 non-null   int64 
 2   title      8692 non-null   object
dtypes: int64(1), object(2)
memory usage: 203.8+ KB


/tmp/ipykernel_2719842/768912109.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yt_df_filtered['trailer_present'] = yt_df_filtered['movieId'].apply(has_trailer)


In [11]:
yt_df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8692 entries, 0 to 8691
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   youtubeId        8692 non-null   object
 1   movieId          8692 non-null   int64 
 2   title            8692 non-null   object
 3   trailer_present  8692 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 271.8+ KB


In [12]:
from pytube import YouTube
from pytube.innertube import _default_clients

_default_clients["ANDROID_MUSIC"] = _default_clients["ANDROID_CREATOR"]

In [13]:
from pytube import YouTube
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

# Assuming yt_df_filtered is your DataFrame
# Add a new column to the DataFrame initialized to 0
yt_df_filtered['video downloaded'] = 0

def download_youtube_video(row):
    video_id = row['youtubeId']
    movie_id = row['movieId']
    trailer_present = row['trailer_present']
    output_path = './yt trailers'

    if(trailer_present==0):
        try:
            video_url = f"https://www.youtube.com/watch?v={video_id}"
            yt = YouTube(video_url)
            # Get the best adaptive video stream (video only, no audio)
            video_stream = yt.streams.filter(adaptive=True, file_extension='mp4').order_by('resolution').desc().first()

            if not video_stream:
                print("No adaptive video stream available for video ID:", video_id)
                return video_id, False

            output_filename = f"{movie_id}.mp4"
            print(f"Downloading best available video for: {yt.title}, resolution: {video_stream.resolution}")
            video_stream.download(output_path, filename=output_filename)
            print("Download complete for video ID:", video_id)
            return video_id, True

        except Exception as e:
            print(f"Error for video ID {video_id}: {str(e)}")
            return video_id, False

if __name__ == "__main__":
    max_workers = 1  # Adjust the number of workers as per your system resources
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(download_youtube_video, row) for _, row in yt_df_filtered.iterrows()]

        for future in as_completed(futures):
            video_id, success = future.result()
            index = yt_df_filtered.index[yt_df_filtered['youtubeId'] == video_id]
            yt_df_filtered.at[index, 'video downloaded'] = 1 if success else 0

    # Save the DataFrame with download statuses to a new CSV file
    yt_df_filtered.to_csv('yt_df_filtered_with_download_status.csv', index=False)

/tmp/ipykernel_2719842/2781333038.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yt_df_filtered['video downloaded'] = 0


Error for video ID ltwvKLnj1B4: ltwvKLnj1B4 is unavailable
Error for video ID SCOxEKkuWG4: SCOxEKkuWG4 is unavailable
Error for video ID a6lGULmQdb0: a6lGULmQdb0 is a private video
Error for video ID JXxFESHwnX0: JXxFESHwnX0 is unavailable
Error for video ID Ns17RQr1yK8: Ns17RQr1yK8 is unavailable
Error for video ID Rieq_TR7cV0: Rieq_TR7cV0 is unavailable
Error for video ID qPPUmzK5pPc: qPPUmzK5pPc is unavailable
Error for video ID RAYuASqrs94: RAYuASqrs94 is unavailable
Error for video ID pg-GMqPHIPQ: pg-GMqPHIPQ is unavailable
Error for video ID zEY1dPRUrVU: zEY1dPRUrVU is unavailable
Error for video ID eETnzbLwTs8: eETnzbLwTs8 is unavailable
Error for video ID -qz_zB2Pc2g: -qz_zB2Pc2g is unavailable
Error for video ID 3Fd6ZlgOFqs: 3Fd6ZlgOFqs is unavailable
Error for video ID mRX2-vM4eYQ: mRX2-vM4eYQ is unavailable
Error for video ID 9J6qOSUJXy0: 9J6qOSUJXy0 is unavailable
Error for video ID Qd_88LUuHGU: Qd_88LUuHGU is unavailable
Error for video ID dxduMVVnrvU: dxduMVVnrvU is unava

KeyboardInterrupt: 

Error for video ID xUnz3llwIjE: xUnz3llwIjE is unavailable
Error for video ID dhOHvdiLaDI: dhOHvdiLaDI is unavailable
Error for video ID bdKOtQlUS6s: bdKOtQlUS6s is age restricted, and can't be accessed without logging in.
Error for video ID OaTEaKhXfzM: OaTEaKhXfzM is unavailable
Error for video ID gszDLDFwcLk: gszDLDFwcLk is age restricted, and can't be accessed without logging in.
Error for video ID I8mWmRSVrbM: I8mWmRSVrbM is unavailable
Error for video ID 4eR7bo9Ertk: 4eR7bo9Ertk is unavailable
Error for video ID cULvtpaIK9I: cULvtpaIK9I is unavailable
Error for video ID vFjjVTrgBmM: vFjjVTrgBmM is unavailable
Error for video ID zr7YfmdIDjU: zr7YfmdIDjU is age restricted, and can't be accessed without logging in.
Error for video ID zj-HQUCgZww: zj-HQUCgZww is unavailable
Error for video ID 6MANr3__teY: 6MANr3__teY is unavailable
Error for video ID NavcqMwqD4w: NavcqMwqD4w is age restricted, and can't be accessed without logging in.Error for video ID tNxozZb7jr4: 'streamingData'
Er

In [14]:
yt_df_filtered

,youtubeId,movieId,title,trailer_present,video downloaded
0,K26_sDKnvMU,1,Toy Story (1995),1,0
1,3LPANjHlPxo,2,Jumanji (1995),1,0
2,rEnOoWs3FuA,3,Grumpier Old Men (1995),1,0
3,j9xml1CxgXI,4,Waiting to Exhale (1995),1,0
4,ltwvKLnj1B4,5,Father of the Bride Part II (1995),0,0
...,...,...,...,...,...
8687,B6uQ5MMlx3g,130960,The Conrad Boys (2006),0,0
8688,qCD9BG7ioOc,130970,George Carlin: Life Is Worth Losing (2005),1,0
8689,zElDJ1f3Jk0,130980,The Day That Lasted 21 Years (2012),1,0
8690,lEqrpuU9fYI,131013,Get Hard (2015),1,0


In [15]:
import os

# Define the path to the folder containing the trailers
trailers_path = "/home/patel8m6/Desktop/python scripts/yt trailers"

# Function to check if the trailer exists
def has_trailer(movie_id):
    # Construct the file name from the movie_id
    file_name = f"{movie_id}.mp4"
    # Generate the full path to the file
    file_path = os.path.join(trailers_path, file_name)
    # Return 1 if the file exists, otherwise 0
    return 1 if os.path.isfile(file_path) else 0

# Apply the function to the movieId column and store the result in a new column
yt_df_filtered['trailer_present'] = yt_df_filtered['movieId'].apply(has_trailer)

# Now yt_df_filtered has an additional column 'trailer_present' with 1 or 0


/tmp/ipykernel_2719842/768912109.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yt_df_filtered['trailer_present'] = yt_df_filtered['movieId'].apply(has_trailer)


In [17]:
# Count the number of 1s and 0s in the 'trailer_present' column
trailer_counts = yt_df_filtered['trailer_present'].value_counts()

# Print the counts
print(trailer_counts)


trailer_present
1    6447
0    2245
Name: count, dtype: int64


In [19]:
yt_df_filtered_2 = yt_df_filtered

In [23]:
yt_df_filtered_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8692 entries, 0 to 8691
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   youtubeId         8692 non-null   object
 1   movieId           8692 non-null   int64 
 2   title             8692 non-null   object
 3   trailer_present   8692 non-null   int64 
 4   video downloaded  8692 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 339.7+ KB


In [24]:
movieId_list_with_trailer = yt_df_filtered_2[yt_df_filtered_2['trailer_present'] == 1]['movieId'].tolist()


In [32]:
yt_only_tralier = yt_df_filtered_2[yt_df_filtered_2['trailer_present'] == 1].copy()


In [33]:
yt_only_tralier.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6447 entries, 0 to 8691
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   youtubeId         6447 non-null   object
 1   movieId           6447 non-null   int64 
 2   title             6447 non-null   object
 3   trailer_present   6447 non-null   int64 
 4   video downloaded  6447 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 302.2+ KB


In [34]:
yt_only_tralier

,youtubeId,movieId,title,trailer_present,video downloaded
0,K26_sDKnvMU,1,Toy Story (1995),1,0
1,3LPANjHlPxo,2,Jumanji (1995),1,0
2,rEnOoWs3FuA,3,Grumpier Old Men (1995),1,0
3,j9xml1CxgXI,4,Waiting to Exhale (1995),1,0
5,2GfZl4kuVNI,6,Heat (1995),1,0
...,...,...,...,...,...
8686,_Buvv8StMGw,130682,Muck (2015),1,0
8688,qCD9BG7ioOc,130970,George Carlin: Life Is Worth Losing (2005),1,0
8689,zElDJ1f3Jk0,130980,The Day That Lasted 21 Years (2012),1,0
8690,lEqrpuU9fYI,131013,Get Hard (2015),1,0


In [35]:
yt_only_tralier.to_csv('yt_only_tralier.csv', index=False)


In [26]:
ratings_small


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [27]:
ratings_small_filtered = ratings_small[ratings_small['movieId'].isin(list(movieId_list_with_trailer))]


In [28]:
ratings_small_filtered

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
...,...,...,...,...
99998,671,6212,2.5,1065149436
99999,671,6268,2.5,1065579370
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [29]:
ratings_small_filtered.reset_index(drop=True, inplace=True)


In [30]:
ratings_small_filtered

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1061,3.0,1260759182
2,1,1129,2.0,1260759185
3,1,1172,4.0,1260759205
4,1,1263,2.0,1260759151
...,...,...,...,...
82357,671,6212,2.5,1065149436
82358,671,6268,2.5,1065579370
82359,671,6365,4.0,1070940363
82360,671,6385,2.5,1070979663


In [31]:
ratings_small_filtered.to_csv('ratings_small_filtered.csv', index=False)
